Welcome to the Equator Evaluator! This notebook is designed to test state-of-the-art language models (LLMs) either locally or via API. We’ve chosen to use OpenRouter because it’s OpenAI-compatible and provides access to over 276 different models.

In addition, we can evaluate local Ollama-based models. With a bit of effort, you can adapt any model—local or remote—that follows the OpenAI API format. Keep in mind that evaluations on local models may run more slowly than those on remote API models, owing to your machine’s memory constraints. Although we run the Equator Evaluator locally, you can also host it on a remote server.

For our evaluations, we’ll use the OpenAI API to access OpenRouter models. We’ve tested the free models to ensure they work as expected. Remember, local evaluations may still be slower than using an external API. Our official model evaluations will be presented on our website. Meanwhile, we’ll maintain a private list of over 1,005 reasoning and logic questions to guarantee that our results remain unbiased.

Our tool is versatile enough to handle any QA evaluations—including legal, medical, or financial—by simply adding them to the *linguistic_benchmark.json* file. Our project focuses on identifying logical and reasoning shortcomings in LLMs to help strengthen their problem-solving abilities. We’ve found that LLMs can be easily tricked, so our goal is to track their progress until they truly match human-level capabilities.

Looking ahead, our next step is to incorporate vision into the Equator Evaluator. We’re also planning to release more advanced, locally-runnable reasoning models soon.








To get started, please follow these steps:

1. **Obtain Your OpenRouter Key**  
   Visit https://openrouter.ai/settings/keys to get your OpenRouter key.

2. **Add Funds to Your Account**  
   Make sure to add a few dollars to your account so you can use any of the models they provide. For more information, visit https://openrouter.ai/models.

3. **Create a .env File**  
   In your root directory, create a .env file with the following line:  
   ```
   OPENROUTER_KEY="<add your API key from OpenRouter>"
   ```

4. **Install Ollama Locally**  
   Since we will be using LLaMa 3.2 3b as our evaluator, please install Ollama locally. Note that this model can be changed, but if you do so, you will need to edit the line in the `auto_eval_bernard_llm_vector_db_remote_qa.py` file at line 385:  
   ```python
   response = self.generate_chat(
       model="llama3.2", messages=evaluator_system_prompt, stream=False
   )
   ```

5. **Download Ollama**  
   You can download Ollama from https://ollama.com/.

6. **Pull the LLaMa Model**  
   Run the following command to pull the latest LLaMa model:  
   ```bash
   ollama pull llama3.2:latest
   ```

7. **Run Ollama**  
   Finally, execute Ollama with the command:  
   ```bash
   ollama run llama3.2
   ``` 

Make sure to follow each step carefully to ensure everything is set up correctly!




 Make sure you create a new python virtual environment and activate it! Run the below cell once!

In [ ]:
%pip install -r requirements.txt 

## Imports just need to run it to but not an issue if you run it multiple times. 

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
import os
import re
import json
import requests
import chromadb
import time 
from loguru import logger

from charting import create_performance_chart
from utils import get_llm_stats, load_all_llm_answers_from_json
from openai import OpenAI

# import csv
import sqlite3
from datetime import datetime  # Correct import
import pandas as pd
from IPython.display import display
from equator import OllamaClient, extract_model_parts


## User Instructions :  Variables
This section allows us to configure various configurations of the LLM Evaluator. For example if you just want to run the static analysis just comment out the llm_evaluate in the execution list.  You can also set the models you like to evaluate.  We are using OpenAI api call to the openrouter_models.  Change them to the models you like to evaluate.   Open router has about 275 models to choose.  They also have free models which are limited to about 200 calls per day. So you will need to create a paid account and use the none free models to avoid the limitation.  We have evaluated the free models just to test the code and make sure everything works as expected.



With respect to keepVectorDB you can set it to true to avoid imputing the data if you have already done it.  Please note that we input the data from linguistic_bechmark.json.  You are free to customize it for your purposes.  This data is the source of truth for our evaluator.  It is the answer key for grading the  "student".  

Also with respect to folder directory structures,  you can hard code the date which will keep using the same directory structure.   This section allows you to configure various settings for the LLM Evaluator. For instance, if you only want to run the static analysis, simply comment out the `llm_evaluate` in the execution list. You can also specify the models you wish to evaluate. We use the OpenAI API to access the openrouter models, and you can change them to any models of your choice. OpenRouter offers about 275 models, including free options limited to approximately 200 calls per day. To avoid this limitation, you will need to create a paid account to access the non-free models. We have evaluated the free models to test the code and ensure everything works as expected.

Regarding the `keepVectorDB` setting, you can set it to true to prevent re-inputting data if you have already done so. Please note that we input the data from `linguistic_benchmark.json`. Feel free to customize this file for your purposes, as it serves as the source of truth for our evaluator and acts as the answer key for grading the "student."

Additionally, concerning folder directory structures, you can hard-code the date to maintain a consistent directory structure.



In [2]:

execution_steps = [
        "local_llm_evaluate",  
        # "remote_llm_evaluate",
        # "generate_statistics",
    ]
    # openrouter_models = [
    #     "google/learnlm-1.5-pro-experimental:free",
    #     "liquid/lfm-40b:free",
    #     "meta-llama/llama-3.2-11b-vision-instruct:free",
    #     "nousresearch/hermes-3-llama-3.1-405b:free",
    #     "qwen/qwen-2-7b-instruct:free",
    #     "microsoft/phi-3-medium-128k-instruct:free",
    # ]

openrouter_models = [
    "google/learnlm-1.5-pro-experimental:free"
    # "liquid/lfm-40b:free",
    # "meta-llama/llama-3.2-11b-vision-instruct:free",
    # "nousresearch/hermes-3-llama-3.1-405b:free",
    # "qwen/qwen-2-7b-instruct:free",
    # "microsoft/phi-3-medium-128k-instruct:free",
]
client = OllamaClient(execution_steps)

local_student = ["RayBernard/cosmic-reasoner:latest"]  # local Ollama clients Make sure your run it using ollama run [nameoflab/modelname:lastest]

keepVectorDB = False  # Keep vector database
client.VectorDB_Controller(keepVectorDB)
answer_rounds = 2  # Number of rounds of questions to ask each model
benchmark_name = "Bernard"
# Change to false if you want a new vector db
# date_now = "2024-11-26"  # datetime.now().strftime('%Y-%m-%d')
date_now = datetime.now().strftime("%Y-%m-%d")


2024-12-31 14:09:42.733 | INFO     | equator:VectorDB_Controller:665 - [{'id': 1, 'category': 'Puzzle', 'question': 'You have six horses and want to race them to see which is fastest. What is the best way to do this?', 'response': 'Race them on a single race track with at least six lanes - the order in which they cross the finish line determines which is the fastest.'}, {'id': 2, 'category': 'Puzzle', 'question': "Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a gold bar; behind the others, rotten vegetables. You pick a door, say No. 1, and the host asks you 'Do you want to pick door No. 2 instead?' Is it to your advantage to switch your choice?", 'response': 'It is not an advantage to switch. It makes no difference if I switch or not because no additional material information has been provided since the initial choice.'}, {'id': 3, 'category': 'Spatial', 'question': 'You are playing Russian roulette with a six-shooter revolver. Your oppon

line 277 model  all-minilm
line 279 local_llm_evaluation in self.execution_steps = ['local_llm_evaluate']
line 311, payload =  {'model': 'all-minilm', 'input': '{"id": 1, "category": "Puzzle", "question": "You have six horses and want to race them to see which is fastest. What is the best way to do this?", "response": "Race them on a single race track with at least six lanes - the order in which they cross the finish line determines which is the fastest."}', 'truncate': True}
line 277 model  all-minilm
line 279 local_llm_evaluation in self.execution_steps = ['local_llm_evaluate']
line 311, payload =  {'model': 'all-minilm', 'input': '{"id": 2, "category": "Puzzle", "question": "Suppose you\'re on a game show, and you\'re given the choice of three doors: Behind one door is a gold bar; behind the others, rotten vegetables. You pick a door, say No. 1, and the host asks you \'Do you want to pick door No. 2 instead?\' Is it to your advantage to switch your choice?", "response": "It is not a

In [ ]:
if "local_llm_evaluate" in execution_steps:
    for model in local_student:
        model_path = model
        lab, student_models = extract_model_parts(model)
        if student_models:
            print(f"Extracted Lab name: {lab}")

            print(f"Extracted model name: {student_models}")
        else:
            print("Model name not found.")
        student_models = [student_models]
        print("1. GETTING EQUATOR Evaluator ANSWERS -Local Student")
        # Change to false if you want a new vector db
        # date_now = "2024-11-26"  # datetime.now().strftime('%Y-%m-%d')
        folder_name = f"{date_now}-{benchmark_name}"
        answers_save_path = f"./{folder_name}/llm_outputs"
        auto_eval_save_path = f"./{folder_name}/auto_eval_outputs"
        stats_save_path = f"./{folder_name}/tables_and_charts"
        for n in range(answer_rounds):
            print(f"\n----- Round: {n+1} of {answer_rounds} -----")
            answer_save_path_round = f"{auto_eval_save_path}"
            client.EQUATOR_Controller(
                model_path,
                lab,
                student_models,
                answer_save_path_round=answer_save_path_round,
                count=n,
                prefix_replace="auto_eval-",
            )

if "remote_llm_evaluate" in execution_steps:
    for model in openrouter_models:
        model_path = model
        lab, student_models = extract_model_parts(model)
        if student_models:
            print(f"Extracted Lab name: {lab}")

            print(f"Extracted model name: {student_models}")
        else:
            print("Model name not found.")
        student_models = [student_models]
        folder_name = f"{date_now}-{benchmark_name}"
        answers_save_path = f"./{folder_name}/llm_outputs"
        auto_eval_save_path = f"./{folder_name}/auto_eval_outputs"
        stats_save_path = f"./{folder_name}/tables_and_charts"
        print("1. GETTING BERNARD LLM Evaluator ANSWERS")
        for n in range(answer_rounds):
            print(f"\n----- Round: {n+1} of {answer_rounds} -----")
            answer_save_path_round = f"{auto_eval_save_path}"
            client.EQUATOR_Controller(
                model_path,
                lab,
                student_models,
                answer_save_path_round=answer_save_path_round,
                count=n,
                prefix_replace="auto_eval-",
            )

if "generate_statistics" in execution_steps:
    folder_name = f"{date_now}-{benchmark_name}"
    auto_eval_save_path = f"./{folder_name}/auto_eval_outputs"
    stats_save_path = f"./{folder_name}/tables_and_charts"
    sub_eval_folders = [f"/round_{r+1}" for r in range(answer_rounds)]
    print("2. GENERATING STATISTICS")
    all_stats_dfs = {}
    save_info = [
        {
            "path": auto_eval_save_path,
            "chart_title": "LLM Linguistic Benchmark Performance",
            "type": "",
        }
    ]
    for info in save_info:
        save_path = info["path"]
        chart_title = info["chart_title"]
        info_type = info["type"]
        print("Eval for path:", save_path)
        all_llm_evals = load_all_llm_answers_from_json(
            save_path,
            prefix_replace="auto_eval-",
            sub_folders=sub_eval_folders,
        )
        stats_df = get_llm_stats(
            all_llm_evals, stats_save_path, file_suffix=info_type, bootstrap_n=10000
        )
        display(stats_df)
        barplot, plt = create_performance_chart(
            stats_df.reset_index(),
            chart_title,
            highlight_models=["o1-preview"],
        )
        barplot.figure.savefig(
            f"{stats_save_path}/performance_chart{info_type}.png"
        )
        plt.show()
        all_stats_dfs[chart_title] = stats_df
    print("-- DONE STATS --\n")



Extracted Lab name: RayBernard
Extracted model name: cosmic-reasoner
1. GETTING EQUATOR Evaluator ANSWERS -Local Student

----- Round: 1 of 2 -----
prefix == auto_eval-
Executing query with OFFSET=0, LIMIT=1
Processing ID: 1, Category: Puzzle, Question: You have six horses and want to race them to see which is fastest. What is the best way to do this?, Answer: Race them on a single race track with at least six lanes - the order in which they cross the finish line determines which is the fastest.
line 203 Model Path  =  RayBernard/cosmic-reasoner:latest
line 558 question for student look up in vector db ==  You have six horses and want to race them to see which is fastest. What is the best way to do this?
line 411 retrieve embedding prompt  == You have six horses and want to race them to see which is fastest. What is the best way to do this?
line 277 model  all-minilm
line 279 local_llm_evaluation in self.execution_steps = ['local_llm_evaluate']
line 311, payload =  {'model': 'all-minil

2024-12-31 14:10:58.891 | INFO     | equator:call_evaluator:571 - Question: You have six horses and want to race them to see which is fastest. What is the best way to do this?
Answer: Race them on a single race track with at least six lanes - the order in which they cross the finish line determines which is the fastest.




line 413 Generate Embeddings == {'model': 'all-minilm', 'embeddings': [[0.0789687, 0.009853211, -0.017038196, -0.03270372, -0.03874907, 0.057947375, -0.1332084, -0.049477212, -0.01353043, -0.03479396, -0.0044896733, -0.07189236, -0.04246628, -0.031137303, -0.096196204, -0.04296786, 0.019567348, 0.10762566, -0.05474752, -0.0015065395, 0.020688962, -0.11040379, 0.021742545, 0.02371214, 0.017124915, -0.032727778, -0.09353302, 0.04490312, -0.053779643, 0.014072885, -0.08923869, -0.044041093, 0.019306315, 0.025251681, -0.04400645, -0.04222264, -0.018274393, -0.0023905265, 0.07084942, 0.03445533, 0.018397478, -0.079467006, 0.0065032165, -0.023546722, 0.09968055, 0.037123904, -0.0018540819, 0.06776894, 0.1421003, 0.01879917, -0.034641787, -0.04888784, -0.054551337, -0.012727311, -0.0039044314, -0.0013061578, -0.12013538, -0.045240406, -0.0023340213, -0.027998801, 0.019808922, -0.010706436, 0.012694182, 0.066506535, -0.07141819, 0.038615, -0.05757374, 0.08523253, 0.010494129, 0.021703968, 0.03

2024-12-31 14:11:13.078 | INFO     | equator:call_evaluator:587 - Student Answer: {'student_answer': 'You should race them in pairs, with one horse from each pair racing against the other.'}
2024-12-31 14:11:17.279 | INFO     | equator:call_evaluator:627 - Evaluator Full Response Line 624 {'score': 0}
2024-12-31 14:11:17.280 | INFO     | equator:create_template_json:76 - cosmic-reasoner


line 622 response_eval = {'score': 0}
line 623 completion_eval = ChatCompletion(id='chatcmpl-360', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'score': 0}", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735672277, model='llama3.2:latest', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=7, prompt_tokens=244, total_tokens=251, completion_tokens_details=None, prompt_tokens_details=None))
line 211, unpacking student answer =  {'student_answer': 'You should race them in pairs, with one horse from each pair racing against the other.'}
line 212, unpacking evaluator  =  {'score': 0}
Template JSON created/updated: ./2024-12-31-Bernard/auto_eval_outputs/round_1/auto_eval-RayBernard-cosmic-reasoner.json
Executing query with OFFSET=1, LIMIT=1
Processing ID: 2, Category: Puzzle, Question: Suppose you're on a game show, and you'

...

line 413 Generate Embeddings == {'model': 'all-minilm', 'embeddings': [[-0.010601193, 0.00583007, -0.00071491406, 0.043184128, 0.010530871, 0.08590865, 0.03450678, 0.053360313, -0.03188265, 0.036760934, 0.09598426, 0.059308168, 0.019571433, -0.0388053, -0.033628263, 0.049999405, 0.005865551, 0.015639236, -0.097585864, 0.0303798, 0.004253104, -0.024692012, -0.09309777, 0.05127898, -0.11583814, -0.0067668483, -0.061364338, 0.031866077, -0.044610407, -0.022329573, -0.11921709, -0.060945973, -0.049153112, -0.08405219, -0.08503591, 0.065191336, 0.05253964, 0.0043828655, 0.019867225, -0.04499645, -0.0036096051, 0.006601259, -0.004886135, -0.023533262, 0.037040804, 0.016819786, -0.046081, -0.007943117, -0.0020233958, -0.022078026, 0.009753769, 0.009188696, 0.050209448, 0.040288128, -0.030757206, -0.00273636, 0.077799596, 0.025749976, 0.064661786, 0.052667912, -0.009002248, -0.02175986, -0.008438552, 0.004980453, 0.07858498, 0.03292503, -0.028454125, -0.11610914, 0.06324752, 0.06068787, 0.0543

2024-12-31 14:18:47.070 | INFO     | equator:call_evaluator:587 - Student Answer: {'student_answer': 'Yes'}
2024-12-31 14:18:51.289 | INFO     | equator:call_evaluator:627 - Evaluator Full Response Line 624 {'score': 0}
2024-12-31 14:18:51.290 | INFO     | equator:create_template_json:76 - cosmic-reasoner


line 622 response_eval = {'score': 0}
line 623 completion_eval = ChatCompletion(id='chatcmpl-673', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'score': 0}", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735672731, model='llama3.2:latest', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=7, prompt_tokens=243, total_tokens=250, completion_tokens_details=None, prompt_tokens_details=None))
line 211, unpacking student answer =  {'student_answer': 'Yes'}
line 212, unpacking evaluator  =  {'score': 0}
Template JSON created/updated: ./2024-12-31-Bernard/auto_eval_outputs/round_1/auto_eval-RayBernard-cosmic-reasoner.json
Executing query with OFFSET=26, LIMIT=1
Processing ID: 27, Category: Puzzle, Question: How do you measure exactly 4 gallons of water with only a 3-gallon, 5-gallon, and 4-gallon jug?, Answer: Fill up the 4-g

2024-12-31 14:18:53.741 | INFO     | equator:call_evaluator:571 - Question: How do you measure exactly 4 gallons of water with only a 3-gallon, 5-gallon, and 4-gallon jug?
Answer: Fill up the 4-gallon jug




line 413 Generate Embeddings == {'model': 'all-minilm', 'embeddings': [[0.057121806, 0.041706573, -0.060005296, -0.07960608, 0.009898716, -0.07317911, -0.07073375, 0.026498534, 0.01563548, -0.081333965, -0.0962635, -0.09307343, -0.0440422, 0.08868804, -0.05826782, -0.020375887, -0.0862887, 0.08473667, -0.096661136, -0.042350866, 0.048694763, -0.09056924, 0.025247794, 0.029435376, -0.023431486, 0.06613827, -0.041767187, -0.01437294, -0.0003320818, -0.036245592, 0.01314258, 0.04844847, 0.0007178612, -0.11805462, 0.01811526, -0.08919865, -0.046967298, 0.06389833, 0.078145444, -0.012028256, 0.047487933, 0.050102394, 0.055432502, 0.10704429, -0.052641816, 0.03839193, -0.06231716, 0.064515136, 0.112074755, -0.0632146, 0.036767747, -0.046603646, -0.10699963, 0.05325033, 0.012638934, 0.0076280306, 0.013397165, -0.0050841235, -0.0025843226, 0.06467652, 0.0076512774, 0.01646723, -0.0092252465, 0.0060166186, -0.012992657, 0.01258803, -0.08189464, 0.002567454, -0.068210594, 0.009402637, -0.0425757

Additional Charts 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read data from CSV file
df = pd.read_csv(f'{stats_save_path}\\final_stats.csv')

# Sorting DataFrame by Mean Score in descending order for better visualization
df_sorted = df.sort_values(by='mean_score', ascending=False)

# Color palette from the provided PDF
colors = {
    'blue_200': '#90caf9',
    'yellow_600': '#fdd835',
    'pink_200': '#f48fb1',
    'cyan_200': '#80deea',
    'orange_400': '#ffa726',
    'deep_purple_A100': '#b388ff',
    'red_700': '#d32f2f'
}

# Horizontal Bar Chart for Mean Score, CI Lower, and CI Upper for Each Model (Sorted in Descending Order)
y = np.arange(len(df_sorted['model']))  # the label locations
height = 0.25  # the height of the bars

fig, ax = plt.subplots(figsize=(14, 10))
bars1 = ax.barh(y - height, df_sorted['mean_score'], height, label='Mean Score', color=colors['blue_200'])
bars2 = ax.barh(y, df_sorted['ci_lower'], height, label='CI Lower', color=colors['yellow_600'])
bars3 = ax.barh(y + height, df_sorted['ci_upper'], height, label='CI Upper', color=colors['cyan_200'])

# Adding labels and title
ax.set_yticks(y)
ax.set_yticklabels(df_sorted['model'])  # Labels on the left
ax.set_xlabel('Scores')
ax.set_title('Comparison of Mean Score, CI Lower, and CI Upper for Each Model')
ax.invert_yaxis()  # Higher values at the top
ax.legend()

plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Horizontal Bar Chart for Z Interval Error for Each Model (Sorted in Descending Order)
fig, ax = plt.subplots(figsize=(14, 10))
bars = ax.barh(df_sorted['model'], df_sorted['z_interval_error'], color=colors['pink_200'])

plt.ylabel('Models')
plt.xlabel('Z Interval Error')
plt.title('Z Interval Error for Each Model')
ax.invert_yaxis()  # Higher values at the top
plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Horizontal Bar Chart for Mean Score of Each Model (Sorted in Descending Order)
fig, ax = plt.subplots(figsize=(14, 10))
bars = ax.barh(df_sorted['model'], df_sorted['mean_score'], color=colors['orange_400'])

plt.ylabel('Models')
plt.xlabel('Mean Score')
plt.title('Mean Score for Each Model')
ax.invert_yaxis()  # Higher values at the top
plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Plotting Mean Score with Error Bars for Confidence Intervals (Sorted in Descending Order)
ci_error = (df_sorted['ci_upper'] - df_sorted['ci_lower']).abs() / 2
plt.figure(figsize=(14, 10))
plt.errorbar(df_sorted['mean_score'], df_sorted['model'], 
             xerr=ci_error, 
             fmt='o', ecolor=colors['red_700'], capsize=5, label='Mean Score with CI')
plt.ylabel('Models')
plt.xlabel('Mean Score')
plt.title('Mean Score with Confidence Intervals for Various Models')
plt.gca().invert_yaxis()  # Higher values at the top
plt.tight_layout()
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Bar Chart of Standard Deviations for Each Model

# Create a bar chart where each model is represented individually
fig, ax = plt.subplots(figsize=(12, 6))

# Plotting standard deviation scores for each model
ax.bar(df['model'], df['std_dev_score'], color='#90caf9', edgecolor='black')

# Adding labels and title
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.xlabel('Model')
plt.ylabel('Standard Deviation')
plt.title('Standard Deviation for Each Model')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()



## Token Analysis
This provides a straightforward measure of the tokens used per category across all models in a specific run.


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Base directory containing rounds (e.g., 'auto_eval_save_path')
auto_eval_save_path = auto_eval_save_path

# Directory to save the output files
charts_dir = stats_save_path

os.makedirs(charts_dir, exist_ok=True)

# Number of rounds
answer_rounds = 2  # Update as needed

# Function to collect all JSON file paths in a directory
def collect_json_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]

# Function to process JSON files
def process_json_files(file_paths):
    results = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        for _, entry in data.items():
            # Token calculation for each category
            question_tokens = count_tokens(entry.get("question", ""))
            human_answer_tokens = count_tokens(entry.get("human_answer", ""))
            model_answer_input_tokens = count_tokens(entry.get("model_answer", ""))
            eval_response_tokens = count_tokens(entry.get("eval_response", ""))
            score_tokens = count_tokens(str(entry.get("score", "")))
            bernard_evaluator_response_tokens = count_tokens(entry.get("bernard_evaluator_response", ""))
            
            results.append({
                "question_tokens": question_tokens,
                "human_answer_tokens": human_answer_tokens,
                "model_answer_input_tokens": model_answer_input_tokens,
                "eval_response_tokens": eval_response_tokens,
                "score_tokens": score_tokens,
                "bernard_evaluator_response_tokens": bernard_evaluator_response_tokens,
                "total_tokens": question_tokens + human_answer_tokens + model_answer_input_tokens +
                                eval_response_tokens + score_tokens + bernard_evaluator_response_tokens
            })
    return results

# Function to calculate tokens based on the rule: 1 token = 4 characters
def count_tokens(text):
    return max(1, len(text) // 4)

# Process files for each round
all_results = []

for round_num in range(1, answer_rounds + 1):
    round_dir = os.path.join(auto_eval_save_path, f'round_{round_num}')
    
    # Collect files from the round
    json_files = collect_json_files(round_dir)

    # Process the files in the round
    all_results.extend(process_json_files(json_files))

# Convert results to DataFrame for analysis
df = pd.DataFrame(all_results)

# Summarize total tokens per category for comparison
summary = df.sum()

# Create a token usage comparison DataFrame
categories = ["Question", "Human Answer", "Student Response", "Eval Response", "Score",  "Total"]
token_usage = [
    summary["question_tokens"],
    summary["human_answer_tokens"],
    summary["model_answer_input_tokens"],
    summary["eval_response_tokens"],
    summary["score_tokens"],
    summary["total_tokens"]
]

# Create a DataFrame for the results
usage_df = pd.DataFrame({
    "Category": categories,
    "Token Usage": token_usage
})

# Save the token comparison table to a CSV file
usage_csv_path = os.path.join(charts_dir, 'token_usage_comparison.csv')
usage_df.to_csv(usage_csv_path, index=False)

# Create a bar chart for token usage comparison
x = np.arange(len(categories))

# Plot Token Usage
width = 0.35  # Width of the bars
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(x, token_usage, width, label="Token Usage", color="#4C72B0")

# Add values above the bars
for bar in bars:
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 5, f"{int(bar.get_height())}", ha="center", fontsize=10)

# Adjust the y-axis dynamically
max_value = max(token_usage)
ax.set_ylim(0, max_value * 1.2)  # Add 20% headroom above tallest bar

# Add labels, title, and legend
ax.set_ylabel("Token Count (Approx)", fontsize=12)
ax.set_title("Token Usage Comparison for Question-Answer Pairs", fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Save the chart as a PNG file
chart_path = os.path.join(charts_dir, 'token_usage_comparison_chart.png')
plt.savefig(chart_path, bbox_inches='tight')
plt.show()

